In [53]:
from Crawler import Crawler
from ParsePubMed import ExceptionHandler
from datetime import datetime
from PubMedIdTranslator import PubMedIdTranslator
from Questions import Questions
from LLMapi import LLMapi

### Jak to ma działać

- aplikacja w trybie tekstowym
- na starcie dostaje listę (w postaci pliku lub listy przecinkowej) DOI, które prowadzą do serii artykułów, które należy zbadać
- klasa PubMedCrawler obsługuje popjedynczy artykuł
- klasa PubMedCrawler wykonuje następujące czynności: parsuje artykuł, odnotowuje w bazie MongoDB wszystkie informacje o artykule, tworzy nową serię wątków dla kolejnych pmid`s, odpytuje LLM API i zapisuje odpowiedzi w bazie
- crawler przeszukuje artykuły po pmid`s do 100 skoków od artykułu bazowego

In [2]:
# Crawler.DB_connector.collection.delete_many({})
len(list(Crawler.DB_connector.collection.find({})))

426

In [2]:
with open('Articles_list_dois.csv') as dois:
    dois = [doi.strip() for doi in dois.readlines()]

In [3]:
dois += '''10.1016/j.scitotenv.2023.169774 
10.1002/edn3.475
10.1016/j.ecolind.2023.110898
10.1002/edn3.413
10.1016/j.scitotenv.2023.161970
10.1111/mec.16737
10.1016/j.scitotenv.2022.155175 
10.3897/mbmg.5.74096
10.1111/1755-0998.13568
10.1016/j.scitotenv.2020.144208
10.1016/j.ecolind.2019.105775
10.1111/jpy.12886
10.1080/0269249X.2023.2237977
10.1002/ece3.8972
10.1002/edn3.452
10.1002/edn3.193
10.7717/peerj.11057
10.1038/s43705-022-00118-3
10.1007/s10750-021-04718-3
10.3389/fmicb.2019.00261
10.3897/mbmg.4.51433
10.1002/edn3.390
10.1186/s12862-023-02118-w
10.1016/j.scitotenv.2019.04.247
10.1038/s41598-020-73739-0
10.1111/fwb.13487
10.1007/s10531-023-02686-9
10.1111/1755-0998.13954
10.1016/j.jglr.2024.102344
10.1111/jbi.14822
10.3389/fmars.2023.1275320
10.1016/j.ecohyd.2023.07.005
10.1038/s41598-022-15442-w
10.1098/rspb.2022.0162
10.1038/s41598-021-93859-5
10.1016/j.biocon.2014.11.029
10.3389/fmars.2021.704786
10.5061/dryad.j9kd51cbr
10.1002/edn3.358
10.1111/mec.15620
10.1016/j.scitotenv.2021.149030
10.1016/j.biocon.2023.110089
10.1111/1755-0998.13610
10.1007/s10750-020-04260-8
10.1002/edn3.443
10.1002/eap.2457
10.1002/edn3.125
10.1002/edn3.130
10.1111/1755-0998.13012
10.3897/BDJ.8.e55137
10.3897/BDJ.11.e97484
10.1080/01650424.2019.1571612
10.3897/BDJ.9.e69841
10.3897/zookeys.1168.101334
10.1139/gen-2019-0226
10.1371/journal.pone.0108651
10.3897/zookeys.592.8316
10.3897/dez.67.56163
10.1038/s41598-024-56930-5
10.1371/journal.pone.0298025
10.1371/journal.pone.0161664
10.3897/mbmg.7.95650
10.1007/s10531-024-02833-w
10.1002/rra.4265
10.1038/s41598-017-15823-6
10.1139/gen-2015-0220
10.1016/j.ecss.2020.106683
10.1016/j.rsma.2020.101341
10.3897/mbmg.5.70063
10.1007/s00027-021-00828-1
10.1016/j.ecss.2022.108207
10.3390/d15050657
10.3390/ani13243876
10.1101/2024.01.25.577180
10.1016/j.scitotenv.2024.171849
10.1016/j.scitotenv.2020.141969
10.1016/j.ecolind.2020.106383
10.1186/s12302-018-0157-x
10.1111/1755-0998.12940
10.3897/mbmg.5.66557
10.3897/mbmg.4.51433
10.1002/edn3.372
10.1007/s10750-021-04718-3
10.1111/icad.12710
10.1002/rra.4265
10.1016/j.marenvres.2023.106175
10.1186/s12866-023-03161-9
10.1016/j.aquaculture.2023.739249
10.1093/femspd/ftad025
10.1093/femsec/fiac141
10.7717/peerj.3687
10.1016/j.margen.2016.01.005 
10.1016/j.mex.2021.101238
10.1038/srep02471
10.1111/MEC.12985
10.1111/1462-2920.12532
10.1093/femsec/fiv086
10.3389/fmicb.2017.01151
10.3389/fmicb.2017.02131
10.1186/s40168-018-0513-5
10.1111/1462-2920.14377
10.3389/fmicb.2019.00908
10.3390/microorganisms9102077
10.1038/s41396-022-01282-z
10.1186/s12915-022-01379-z
10.1128/spectrum.01112-23
10.1002/lno.12429
10.1002/lno.12437
10.3390/jof9111095
10.1371/journal.pone.0236540
10.1126/sciadv.abf5908
10.1111/jeu.12700
10.1080/09670262.2011.554886
10.3390/v10120715
10.1093/ismeco/ycae014
10.3389/fmars.2022.824100
'''.split()

In [4]:
def crawl(DOIs:list=None, allowed_steps:int=None, DEBUG:bool=True,log=None) -> int:
    ExceptionHandler.checkType(DOIs,'DOIs',list)
    for doi in DOIs:
        ExceptionHandler.checkType(doi,'element of DOIs',str)
    ExceptionHandler.checkType(allowed_steps,'allowed_steps',int)
    ExceptionHandler.checkType(DEBUG,'DEBUG',bool)

    for doi in DOIs:
        try:
            crawler = Crawler(doi,list(),allowed_steps,DEBUG,log=log)
            crawler.request()
        except Exception as e:
            log.write(f'{datetime.now()} [Error] {doi}: Something went wrong. Error message: {e}\n')
            if DEBUG: print(f'Something went wrong on path: {doi}. Error message: {e}')

In [ ]:
with open('log.txt','a') as log:
    status = crawl(dois,20,log=log)
print(status)

10.1371/journal.pone.0238557: Paper already in database
Something went wrong on path: 10.1371/journal.pone.0238557. Error message: Paper already in database
10.1038/s41598-021-85488-9: Paper already in database
Something went wrong on path: 10.1038/s41598-021-85488-9. Error message: Paper already in database
10.1038/s41598-022-15488-w: Paper already in database
Something went wrong on path: 10.1038/s41598-022-15488-w. Error message: Paper already in database
10.7717/peerj.14071: Paper already in database
Something went wrong on path: 10.7717/peerj.14071. Error message: Paper already in database
10.1371/journal.pone.0266720: Paper already in database
Something went wrong on path: 10.1371/journal.pone.0266720. Error message: Paper already in database
10.1002/ece3.4213: Paper already in database
Something went wrong on path: 10.1002/ece3.4213. Error message: Paper already in database
10.1371/journal.pone.0236540: Paper already in database
Something went wrong on path: 10.1371/journal.pone

10.1111/1755-0998.13847: DOI translated
Connecting... 200
10.1111/1755-0998.13847: Cannot process given ID
10.1111/1755-0998.13847: PubMed didn`t responded well. Trying with PubMed Central...
10.1111/1755-0998.13847: DOI translated
Connecting... 200
10.1111/1755-0998.13847: Cannot process given ID
10.1111/1755-0998.13847: PubMed Central didn`t responded well. Trying with SciHub...
Creating temporary PDF file
Requesting for article...
Connecting... 
200
Extracting PDF URI
Requesting for PDF
Extracting text from PDF
Cannot find PDF for given DOI
Parsing article...
10.1111/1755-0998.13847: Cannot process given ID: No text to parse. First run .request() method.
Deleting temporary PDF file
10.1111/1755-0998.13847: Cannot find given DOI
Something went wrong on path: 10.1111/1755-0998.13847. Error message: Cannot find given DOI: 10.1111/1755-0998.13847
10.1038/s41598-023-32862-4: Paper already in database
Something went wrong on path: 10.1038/s41598-023-32862-4. Error message: Paper already i

In [7]:
for i in Crawler.DB_connector.collection.find():
    if 'Q10' in i.keys():
        print(i['Q10'])
        print('-------------')

The reference database used for taxonomical identification is the local macroinvertebrate library constructed using the Barcode of Life Database (BOLD) and the naive Bayesian method.
-------------
The reference database used for taxonomical identification is "Diat.barcode (formerly called R-Syst::diatom in Rimet et al., 2016)".
-------------
There is no reference database used for taxonomical identification in the given text. Instead, the text discusses the use of DNA-based methods for species detection and the potential for these methods to be used in place of traditional morphology-based methods. The text also mentions the need for robust quality assurance procedures to ensure the accuracy of the results.
-------------
The reference database used for taxonomical identification is not explicitly mentioned in the text. However, based on the context, it seems that the authors are using an automated taxa identification system that relies on computer-based taxa identification from image d

In [23]:
counter = 0
length = 0
with open('Articles_list_pmids.csv') as pmids:
    with open('Articles_list_dois.csv','w') as dois:
        for pmid in pmids:
            length += 1
            try:
                dois.write(PubMedIdTranslator.PubMedtoDOI(int(pmid)) + '\n')
            except TypeError:
                counter += 1

print(f'{counter}/{length} ({100*counter/length:.2f} %) failed')

773/1677 (46.09 %) failed


## Summarizing answears

In [9]:
questions = Questions()

In [76]:
for i,q in enumerate(questions._questions):
    print(f'Q{i+1}: {q}')

Q1: Are the data in this paper environmental? Only "yes" or "no".
Q2: Are the data available in paper or supplement? Only "paper" or "suplement".
Q3: What is the sample collection method?
Q4: What is the DNA extraction method?
Q5: What is the source of the protocol in protocols.io?
Q6: What is the overall sequencing strategy used in experiment?
Q7: What is the sequence analysis workflow?
Q8: Where is the data stored?
Q9: What is the marker name used in experiment?
Q10: What is the reference database used for taxonomical identification?


In [77]:
summarizes = {
    'Q1':None,
    'Q2':None,
    'Q3':'Basing on provided text write overall sample collection method.',
    'Q4':'Basing on provided text write overall DNA extraction method.',
    'Q5':None,
    'Q6':'Basing on provided text write overall sequencing strategy used in experiments.',
    'Q7':'Basing on provided text write overall sequence analysis workflow.',
    'Q8':None,
    'Q9':None,
    'Q10':None
}

In [17]:
resp = list(Crawler.DB_connector.collection.find())

### Question 2

In [61]:
resp_q2 = ['both' if 'both' in r[f'Q2'] else ('paper' if 'paper' in r['Q2'] else 'supplement') for r in resp if f'Q{i}' in r.keys()]

In [62]:
np.uniresp_q2

['supplement',
 'both',
 'supplement',
 'supplement',
 'paper',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'paper',
 'supplement',
 'supplement',
 'paper',
 'paper',
 'paper',
 'paper',
 'supplement',
 'supplement',
 'paper',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'paper',
 'supplement',
 'paper',
 'paper',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'paper',
 'supplement',
 'paper',
 'supplement',
 'supplement',
 'supplement',
 'paper',
 'paper',
 'paper',
 'supplement',
 'paper',
 'paper',
 'paper',
 'supplement',
 'supplement',
 'paper',
 'paper',
 'supplement',
 'supplement',
 'paper',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'supplement',
 'paper',
 'supplement',
 'supplement',
 'paper',
 'supplement',
 'supplement',
 'supplemen

In [54]:
LLM_API:LLMapi = LLMapi(URL='http://10.4.24.103',verbose=True)

Connection ok


In [70]:
LLM_API.ask('Basing on given text write an overall analysis workflow',responses_cumulative['Q7'])['Answer']

'Based on the given text, the overall analysis workflow could be summarized as follows:\n\n1. Data Collection: Gathering hydrographic data along specific lines in the North Pacific Ocean.\n\n2. Preprocessing: Cleaning and preprocessing the data to remove any irrelevant information and format inconsistencies.\n\n3. Tokenization: Breaking down the text into individual words or phrases, known as tokens, to facilitate analysis.\n\n4. Part-of-speech tagging: Tagging each token with its part of speech to provide context and meaning.\n\n5. Named Entity Recognition: Identifying and extracting named entities such as people, places, or organizations.\n\n6. Dependency Parsing: Analyzing the relationships between tokens, such as subject-verb-object relationships, to understand the sentence structure and meaning.\n\n7. Sentiment Analysis: Determining the overall sentiment of the text, whether positive, negative, or neutral.\n\n8. Topic Modeling: Identifying any recurring topics or themes in the tex

In [52]:
responses_cumulative['Q10']

'According to the text, the reference database used for taxonomical identification is the "CRUX-generated 12S reference database supplemented with FishCARD reference sequences."\nThe reference database used for taxonomical identification is a custom-made database created by downloading whole and partial fish mitogenome sequences from MitoFish and whole mitogenome sequences from tetrapods from NCBI Organelle Genome Resources. Additionally, the database was supplemented by assembling new sequences in the author\'s laboratory. As of 4 October 2014, the database covers approximately 4230 fish species distributed across 457\nBased on the text, the reference database used for taxonomical identification is the GenBank Nucleotide database.\nBased on the content of the text, the reference database used for taxonomical identification is GenBank.\nBased on the text, the reference database used for taxonomical identification is "DNA barcodes that enable species identiﬁcation".\n\nNote: DNA barcodi